DOG VS CAT CLASSIFICATION

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [23]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.5,
    rotation_range=15,
    shear_range=0.5,
    horizontal_flip=True,
    fill_mode='nearest'
    

)

test_datagen=ImageDataGenerator(
    rescale=1./255
)

In [24]:
train_generator=train_datagen.flow_from_directory(
    r'/kaggle/input/cats-and-dogs-image-classification/train',
    subset='training',
    class_mode='binary',
    target_size=(150,150)
    
)

Found 557 images belonging to 2 classes.


In [25]:
test_generator=test_datagen.flow_from_directory(
    r'/kaggle/input/cats-and-dogs-image-classification/test',
    class_mode='binary',
    target_size=(150,150)
    

)

Found 140 images belonging to 2 classes.


In [26]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3))

#Extract features from the intermediate layers.
model=Model(inputs=base_model.input,outputs=base_model.get_layer('block5_pool').output)



In [29]:
def extract_features(generator,model,batch_size=32):
    features=[]
    labels=[]
    for inputs_batch,labels_batch in generator:
        features_batch=model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features)*batch_size>=generator.samples:
            break
    return np.vstack(features),np.concatenate(labels)

In [30]:
train_features,train_labels= extract_features(train_generator,model)
test_features,test_labels= extract_features(test_generator,model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step


In [32]:
train_features=train_features.reshape((train_features.shape[0],-1))
test_features=test_features.reshape((test_features.shape[0],-1))

In [33]:
scaler=StandardScaler()
train_features=scaler.fit_transform(train_features)
test_features=scaler.transform(test_features)

In [34]:
svm_model=SVC(kernel='linear')
svm_model.fit(train_features,train_labels)
y_pred=svm_model.predict(test_features)

In [35]:
accuracy=accuracy_score(test_labels,y_pred)
classification=classification_report(test_labels,y_pred)

In [36]:
print("Accuracy: ",accuracy)
print("Classification Report: ")
print(classification)

Accuracy:  0.7428571428571429
Classification Report: 
              precision    recall  f1-score   support

         0.0       0.72      0.79      0.75        70
         1.0       0.77      0.70      0.73        70

    accuracy                           0.74       140
   macro avg       0.74      0.74      0.74       140
weighted avg       0.74      0.74      0.74       140



Thank you!